In [30]:
import openpyxl
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pytz 


def get_pytz_timezone(start_timezone):
    # Mapping friendly time zone names to pytz time zone names
    timezone_mapping = {
        'argentina': 'America/Argentina/Buenos_Aires',
        'pst': 'America/Los_Angeles',
        'pdt': 'America/Los_Angeles',
        'pst/pdt': 'America/Los_Angeles',
        'PST/PDT': 'America/Los_Angeles',
        'CET/CEST': 'Europe/Berlin',
        'cet/cest': 'Europe/Berlin',
        'chile': 'America/Santiago',
        'bst/gmt': 'Europe/London',
        'bst': 'Europe/London',
        'BST': 'Europe/London',
        'colombia': 'America/Bogota',
        'Central Time(US)': 'US/Central',
        #add
        # !CASE sensitive issue, BST not parsing - bst parsing
    }

    # Try to find a direct timezone match
    pytz_timezone = timezone_mapping.get(start_timezone.lower())

    print(f"mapped timezone: {pytz_timezone}")

    return pytz_timezone



def read_single_row(file_path, row_num):
    try:
        # Load the workbook
        workbook = openpyxl.load_workbook(file_path)
        
        # Select the active sheet (you can modify this if you want a specific sheet)
        sheet = workbook.active

        # Check if the specified row exists
        if row_num <= 0 or row_num > sheet.max_row:
            print(f"Row {row_num} does not exist in the sheet.")
            return

        # Get column indices for the specified columns
        description_col = get_column_index(sheet, "Description")
        start_time_col = get_column_index(sheet, "Start Time")
        time_zone_col = get_column_index(sheet, "Time Zone")
        duration_col = get_column_index(sheet, "Duration")

        # Get data from specified columns in the chosen row
        start_description = sheet.cell(row=row_num, column=description_col).value
        start_time = sheet.cell(row=row_num, column=start_time_col).value
        start_timezone = sheet.cell(row=row_num, column=time_zone_col).value
        duration_hours = sheet.cell(row=row_num, column=duration_col).value
        duration_hours = duration_hours.replace(" hours", "")

        print("read_single_row func output")
        
        print(f"Description {start_description}")
        print(f"Start Time {start_time}")
        print(f"Time Zone {start_timezone}")
        print(f"Duration {duration_hours}")
        print("")
        

    except Exception as e:
        print(f"An error occurred: {e}")


def get_column_index(sheet, column_name):
    # Get the column index based on the column name
    for col_num in range(1, sheet.max_column + 1):
        if sheet.cell(row=1, column=col_num).value == column_name:
            return col_num
    raise ValueError(f"Column '{column_name}' not found in the sheet")

target_timezone = 'Europe/London'

if __name__ == "__main__":
    # Get user input for file path and row number
    file_path = input("Enter the Excel file path (with or without quotes): ").strip('\"\'')
    row_num = int(input("Enter the row number to grab data: "))

    # Call the function to read and process data for the chosen row
    read_single_row(file_path, row_num)


Row 50 does not exist in the sheet.
